# Capstone Project - The Battle of Neighborhoods
## Introduction/Business Problem

### Paris neighbourhoods review to find gaps on Coffee shops opportunities. There is an interest on a new openning of coffee shop, so we will analize the population density agains the existing coffee shops to select the top 3 neighbourhoods for this business investment. The target audience is the group of investors when optimizing lowering risk of investment and maximize benefits.

## Data sources for the analysis

### Wikipedia neighbourhoods information, we get Population & Area(Hec): Quartiers of Paris (https://en.wikipedia.org/wiki/Quarters_of_Paris) 
### GeoData Quariers Paris 
#### Quartiers geo structure: https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=geojson&timezone=Europe/Berlin
#### Quartiers geo locations: https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
### Foursquare venues information

## Data capture for the analysis

In [1]:
import pandas as pd
import numpy as np
!pip install folium
import folium as fl
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install wget
import wget
import json

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

### Data  Capture

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Quarters_of_Paris")
df = tables[0][1:]
df.columns = tables[0].iloc[0]
df.is_copy = False
df.drop(df.columns[[5]], axis=1, inplace=True)
df.columns = ['Borough','Neighbourhood', 'Population', 'Area', 'Map']
df[0:10]

,Borough,Neighbourhood,Population,Area,Map
1,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9
2,2nd,Les Halles,8984,41.2,NaN
3,3rd,Palais-Royal,3195,27.4,NaN
4,4th,Place-Vendôme,3044,26.9,NaN
5,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8
6,6th,Vivienne,2917,24.4,NaN
7,7th,Mail,5783,27.8,NaN
8,8th,Bonne-Nouvelle,9595,28.2,NaN
9,"3rd arrondissement(Called ""du Temple"")",9th,Arts-et-Métiers,9560,31.8
10,10th,Enfants-Rouges,8562,27.2,NaN


In [3]:
district = ''
for index, row in df.iterrows():
    if (index % 4 == 1): 
        district = df['Borough'][index]
        df['Neighbourhood'][index] = df['Population'][index]
        df['Population'][index] = df['Area'][index]
        df['Area'][index] = df['Map'][index]
    df['Borough'][index] = district
df.is_copy = False
df.drop(df.columns[[4]], axis=1, inplace=True)
df[0:10]

,Borough,Neighbourhood,Population,Area
1,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9
2,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2
3,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4
4,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9
5,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8
6,"2nd arrondissement(Called ""de la Bourse"")",Vivienne,2917,24.4
7,"2nd arrondissement(Called ""de la Bourse"")",Mail,5783,27.8
8,"2nd arrondissement(Called ""de la Bourse"")",Bonne-Nouvelle,9595,28.2
9,"3rd arrondissement(Called ""du Temple"")",Arts-et-Métiers,9560,31.8
10,"3rd arrondissement(Called ""du Temple"")",Enfants-Rouges,8562,27.2


Calculate density Population/Area(Hec)

In [4]:
df["Population"] = pd.to_numeric(df["Population"])
df["Area"] = pd.to_numeric(df["Area"])
df['Density'] = df['Population']/df['Area']
df.head()

,Borough,Neighbourhood,Population,Area,Density
1,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9,19.240506
2,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2,218.058252
3,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4,116.605839
4,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9,113.159851
5,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8,71.542553


In [5]:
neighborhoods_paris = df
print('Shape After data cleanup:',neighborhoods_paris.shape)

Shape After data cleanup: (80, 5)


### Merge geo data 

In [10]:
from pandas.io.json import json_normalize  
url = 'https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=geojson&timezone=Europe/Berlin'  
wget.download(url, 'paris_quartiers.json')
geo_data = pd.read_json("paris_quartiers.json")
xy_data = json_normalize(data = geo_data['features'])
xy_data.drop(xy_data.columns[[0,1,2,4,6,7,8,9,10,11,12,13]], axis=1, inplace=True)
xy_data.columns = ['cq','lt']
xy_data['cq'] = pd.to_numeric(xy_data['cq'])
xy_data.sort_values(by = ['cq'], ascending=True, inplace=True)
print(xy_data.shape)
xy_data.head()

(80, 2)


,cq,lt
37,1,"[48.86065013520992, 2.334910329280203]"
72,2,"[48.86228910809422, 2.344898858311021]"
23,3,"[48.86465997810258, 2.336308918965308]"
73,4,"[48.86701859062769, 2.328581664931251]"
74,5,"[48.86930663808872, 2.333431807658117]"


In [11]:
idx=1
neighborhoods = pd.DataFrame(columns = ['Borough', 'Neighbourhood', 'Population', 'Area', 'Density', 'Latitude', 'Longitude'])
for hood, borough, population, area, density in zip(neighborhoods_paris['Neighbourhood'],neighborhoods_paris['Borough'],neighborhoods_paris['Population'],neighborhoods_paris['Area'],neighborhoods_paris['Density']):
    location = xy_data.query('cq == @idx')['lt'].values
    #print(location[0][0],'-',location[0][1])
    neighborhoods.loc[len(neighborhoods)] = [borough, hood, population, area, density, location[0][0], location[0][1]]
    idx=idx+1
print(neighborhoods.shape)
neighborhoods.head()

(80, 7)


,Borough,Neighbourhood,Population,Area,Density,Latitude,Longitude
0,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9,19.240506,48.860650,2.334910
1,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2,218.058252,48.862289,2.344899
2,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4,116.605839,48.864660,2.336309
3,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9,113.159851,48.867019,2.328582
4,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8,71.542553,48.869307,2.333432


## Maps data 

In [12]:
address = 'Paris, FR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [13]:
map_data=r'paris_quartiers.json'

# create map of Paris using latitude and longitude values
map_paris = fl.Map(location=[latitude, longitude], zoom_start=13)
fl.Choropleth(geo_data=map_data,
              data=neighborhoods,
              columns=['Neighbourhood', 'Density'],
              key_on='feature.properties.l_qu',
              fill_color='YlOrRd', 
              fill_opacity=0.7, 
              line_opacity=0.2,
              legend_name='Density Rate in Paris'
            ).add_to(map_paris)


In [14]:

# add markers to map
for lat, lng, borough, neighborhood, population, density in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood'], neighborhoods['Population'], neighborhoods['Density']):
    label = '{}, {}, Population:{}, D:{}'.format(neighborhood, borough, population, density)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

##  Foursquare venues data collection 

In [15]:
# The code was removed by Watson Studio for sharing.

Foursquare Credentails
CLIENT_ID: LCLH3TP21APF0YU5LZE2O3ZRP3GWXCYL5Z5P1UECNVBQJVJZ


In [36]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Explore Neighborhoods in Paris

In [37]:
paris_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

#### Let's check the size of the resulting venues dataframe

In [18]:
print(paris_venues.shape)
paris_venues.head()

(5583, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Saint-Germain-l'Auxerrois,48.86065,2.33491,Musée du Louvre,48.860847,2.336440,Art Museum
1,Saint-Germain-l'Auxerrois,48.86065,2.33491,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
2,Saint-Germain-l'Auxerrois,48.86065,2.33491,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
3,Saint-Germain-l'Auxerrois,48.86065,2.33491,"Pavillon des Sessions – Arts d'Afrique, d'Asie...",48.860724,2.332121,Art Museum
4,Saint-Germain-l'Auxerrois,48.86065,2.33491,Mona Lisa | La Joconde,48.860139,2.335337,Exhibit


In [19]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 305 uniques categories.


### Analyze Each Neighborhood

In [20]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighbourhood'] = paris_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]
print(paris_onehot.shape)
paris_onehot.head()

(5583, 306)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Saint-Germain-l'Auxerrois,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Saint-Germain-l'Auxerrois,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Saint-Germain-l'Auxerrois,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Saint-Germain-l'Auxerrois,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Saint-Germain-l'Auxerrois,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
paris_grouped = paris_onehot.groupby('Neighbourhood').mean().reset_index()
print(paris_grouped.shape)
paris_grouped.head()

(80, 306)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Amérique,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0
1,Archives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.010000,0.00,0.0,0.0,0.0,0.0,0.0
2,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.012195,0.00,0.0,0.0,0.0,0.0,0.0
3,Arts-et-Métiers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,...,0.0,0.0,0.03,0.040000,0.01,0.0,0.0,0.0,0.0,0.0
4,Auteuil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in paris_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amérique----
               venue  freq
0  French Restaurant  0.18
1             Bistro  0.09
2             Bakery  0.09
3        Supermarket  0.09
4               Park  0.09


----Archives----
                 venue  freq
0       Clothing Store  0.08
1    French Restaurant  0.07
2          Art Gallery  0.05
3  Japanese Restaurant  0.04
4   Italian Restaurant  0.04


----Arsenal----
               venue  freq
0  French Restaurant  0.16
1              Hotel  0.06
2              Plaza  0.04
3          Gastropub  0.04
4   Tapas Restaurant  0.04


----Arts-et-Métiers----
                 venue  freq
0    French Restaurant  0.09
1                Hotel  0.07
2  Japanese Restaurant  0.06
3         Cocktail Bar  0.04
4           Restaurant  0.04


----Auteuil----
                   venue  freq
0           Tennis Court  0.33
1                 Garden  0.13
2                 Museum  0.07
3  Outdoors & Recreation  0.07
4                 Office  0.07


----Batignolles----
                venue 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = paris_grouped['Neighbourhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amérique,French Restaurant,Pool,Bakery,Plaza,Park,Bus Stop,Supermarket,Bed & Breakfast,Café,Bistro
1,Archives,Clothing Store,French Restaurant,Art Gallery,Japanese Restaurant,Coffee Shop,Bookstore,Italian Restaurant,Tea Room,Burger Joint,Bistro
2,Arsenal,French Restaurant,Hotel,Historic Site,Gastropub,Plaza,Seafood Restaurant,Tapas Restaurant,Pedestrian Plaza,Museum,Thai Restaurant
3,Arts-et-Métiers,French Restaurant,Hotel,Japanese Restaurant,Bar,Italian Restaurant,Restaurant,Cocktail Bar,Wine Bar,Vietnamese Restaurant,Coffee Shop
4,Auteuil,Tennis Court,Garden,Racecourse,French Restaurant,Sporting Goods Shop,Museum,Botanical Garden,Stadium,Office,Outdoors & Recreation


## Cluster Paris  Neighborhoods with Generic Venues (kind of neighbourhoods trends clusters)

Run *k*-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 5

paris_grouped_clustering = paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 3, 4, 1, 3, 1, 3, 3], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = neighborhoods

# merge paris_grouped with paris_data to add latitude/longitude for each neighborhood
paris_merged =paris_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

paris_merged.head() # check the last columns!

,Borough,Neighbourhood,Population,Area,Density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9,19.240506,48.860650,2.334910,1,French Restaurant,Hotel,Café,Plaza,Cosmetics Shop,Historic Site,Japanese Restaurant,Exhibit,Coffee Shop,Bar
1,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2,218.058252,48.862289,2.344899,1,French Restaurant,Pizza Place,Café,Italian Restaurant,Bakery,Wine Bar,Restaurant,Bar,Historic Site,Tea Room
2,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4,116.605839,48.864660,2.336309,3,Japanese Restaurant,Hotel,French Restaurant,Café,Plaza,Ramen Restaurant,Coffee Shop,Historic Site,Bar,Korean Restaurant
3,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9,113.159851,48.867019,2.328582,0,French Restaurant,Hotel,Japanese Restaurant,Chocolate Shop,Jewelry Store,Tea Room,Boutique,Hotel Bar,Pastry Shop,Bookstore
4,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8,71.542553,48.869307,2.333432,0,Hotel,French Restaurant,Japanese Restaurant,Sandwich Place,Italian Restaurant,Plaza,Cocktail Bar,Jewelry Store,Opera House,Coffee Shop


Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = fl.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_data=r'paris_quartiers.json'

# create map of Paris using latitude and longitude values
map_clusters = fl.Map(location=[latitude, longitude], zoom_start=13)
fl.Choropleth(geo_data=map_data,
              data=neighborhoods,
              columns=['Neighbourhood', 'Density'],
              key_on='feature.properties.l_qu',
              fill_color='YlOrRd', 
              fill_opacity=0.7, 
              line_opacity=0.2,
              legend_name='Density Rate in Paris'
            ).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Neighbourhood'], paris_merged['Cluster Labels']):
    label = fl.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fl.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Let's review Cafés density per neighbourhood

Calculate density to a new dataframe

In [78]:
temp = paris_grouped.copy()
temp.drop(temp.columns[1:50], axis=1, inplace=True)
temp.drop(temp.columns[2:300], axis=1, inplace=True)
paris_cafe = paris_cafe.merge(paris_merged)
paris_cafe.head()

,Neighbourhood,Café,Borough,Population,Area,Density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amérique,0.090909,"19th arrondissement(Called ""des Buttes-Chaumont"")",55365,183.6,301.552288,48.881638,2.395440,1,French Restaurant,Pool,Bakery,Plaza,Park,Bus Stop,Supermarket,Bed & Breakfast,Café,Bistro
1,Archives,0.020000,"3rd arrondissement(Called ""du Temple"")",8609,36.8,233.940217,48.859192,2.363205,3,Clothing Store,French Restaurant,Art Gallery,Japanese Restaurant,Coffee Shop,Bookstore,Italian Restaurant,Tea Room,Burger Joint,Bistro
2,Arsenal,0.000000,"4th arrondissement(Called ""de l'Hôtel-de-Ville"")",9474,48.7,194.537988,48.851585,2.364768,1,French Restaurant,Hotel,Historic Site,Gastropub,Plaza,Seafood Restaurant,Tapas Restaurant,Pedestrian Plaza,Museum,Thai Restaurant
3,Arts-et-Métiers,0.030000,"3rd arrondissement(Called ""du Temple"")",9560,31.8,300.628931,48.866470,2.357083,3,French Restaurant,Hotel,Japanese Restaurant,Bar,Italian Restaurant,Restaurant,Cocktail Bar,Wine Bar,Vietnamese Restaurant,Coffee Shop
4,Auteuil,0.000000,"16th arrondissement(Called ""de Passy"")",67967,303.0,224.313531,48.850622,2.252277,4,Tennis Court,Garden,Racecourse,French Restaurant,Sporting Goods Shop,Museum,Botanical Garden,Stadium,Office,Outdoors & Recreation


### Rank clusters to identify the candidate Clusters with investment capacity (lower Café sites vs Density)

In [100]:
paris_ranking = paris_cafes.groupby('Cluster Labels').mean().reset_index()
paris_ranking['Ratio'] = paris_ranking['Density']*(1-paris_ranking['Café'])/10
paris_ranking.sort_values(by = ['Ratio'], ascending=False, inplace=True)
paris_ranking

,Cluster Labels,Café,Area,Density,Latitude,Longitude,Ratio
2,2,0.000000,186.300000,337.879764,48.830359,2.428827,33.787976
3,3,0.038505,106.329167,260.790383,48.864906,2.362714,25.074868
1,1,0.030532,134.257143,247.042634,48.863235,2.340857,23.949982
4,4,0.000000,303.000000,224.313531,48.850622,2.252277,22.431353
0,0,0.029669,103.773684,184.722789,48.854069,2.326158,17.924223


### Let's draw high level estimated candidate areas (Centroid of clusters, euclidean distance is also valid to assess customers walking/metro distance)

In [102]:
print(latitude, longitude)
# create map
map_candidates = fl.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**3 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_data=r'paris_quartiers.json'

# create map of Paris using latitude and longitude values
map_candidates = fl.Map(location=[latitude, longitude], zoom_start=13)
fl.Choropleth(geo_data=map_data,
              data=neighborhoods,
              columns=['Neighbourhood', 'Density'],
              key_on='feature.properties.l_qu',
              fill_color='YlOrRd', 
              fill_opacity=0.7, 
              line_opacity=0.2,
              legend_name='Candidates Centroids in Paris for Café investment'
            ).add_to(map_candidates)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_ranking['Latitude'], paris_ranking['Longitude'], paris_ranking['Ratio'], paris_ranking['Cluster Labels']):
    label = fl.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fl.CircleMarker(
        [lat, lon],
        radius=poi,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_candidates)

map_candidates

48.8566101 2.3514992
